Mounted at /content/drive
/content/drive/MyDrive/Prompt_Semantic
/content/drive/MyDrive/Prompt_Semantic


In [3]:
# Set up the environment
!pip install -r requirements.txt


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-uj5bje7b
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-uj5bje7b
  Resolved https://github.com/huggingface/transformers to commit 746104ba6f0514159d58dc2fb09c887d0e9d4863
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.5 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9614041 sha256=b8647a66dc2c6b2de8b74ab5227defb6b54b896c773214cff3e2be98f180ecae
  Stored in directory: /tmp/pip-ephem-wheel-cache-q73sdd4d/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully b

In [7]:
# !pip install transformers accelerate bitsandbytes>0.37.0

In [ ]:
# !python main.py \
#   --train_date_path


In [4]:
train_path = "/content/drive/MyDrive/Prompt_Semantic/Data/karma-version/sources/"
models_path = "/content/drive/MyDrive/Prompt_Semantic/Data/models-triples/"
ontology_path = "/content/drive/MyDrive/Prompt_Semantic/Data/ontology/Ontologies.json"

In [5]:
from data_loader import generate_data
from prompt_generator import load_ontologies, generate_prompt_without_semantic_typing
from instruct_prompt_generator import load_ontologies, generate_instruct_prompt
train_data, test_data = generate_data(train_path, models_path,  6, 3)
ontology = load_ontologies(ontology_path)

Loading data from [0, 1] files
Loading data from /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/sources/s01-cb.json and /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/models-json/s01-cb-model.json
Loading data from /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/sources/s02-dma.json and /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/models-json/s02-dma-model.json
Loading data from [2] files
Loading data from /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/sources/s03-ima-artists.json and /content/drive/MyDrive/Prompt_Semantic/Data/karma-version/models-json/s03-ima-artists-model.json


In [ ]:
import os
import json

#The system can add number to the semantic types to distinguish the different semantic type nodes with the same name. For example: crm:E42_Identifier1, crm:E42_Identifier2, crm:E42_Identifier3 represent different semantic type nodes.
def generate_prompt_chain(train_data, test_data, model, ontologies, with_inst=False):
    """Generate a prompt chain for the LLM based on the table data and ontologies."""
    system_prompt = "You are a helpful assistant that can generate semantic graphs for tables based on the ontologies given(delimited by '<Ontology>' and '</Ontology>') and the table data given(delimited by '<Table>' and '</Table>'). \
        In the Ontology part, nodes is delimited by '<Nodes>' and '</Nodes>', properties is delimited by '<Properties>' and '</Properties>', and potential triples is delimited by '<PotentialTriples>' and '</PotentialTriples>'. The symbol '->' in the ontology is used to represent the 'is parent class of' relation. \
        In the Table part, each table is a list of dictionaries, where each dictionary represents a row of the table, and the key-value pairs are the column names and their corresponding values."
    nodes = []
    properties = []
    potential_triples = []
    for ontology_name, ontology_data in ontologies.items():
        for node in ontology_data['Nodes']:
            nodes.append(node)
        for prop in ontology_data['Properties']:
            properties.append(prop)
        for triples in ontology_data['Potential triples']:
            potential_triples.append(triples.replace("<", "[").replace(">", "]"))
    system_prompt += f"<Ontology> <Nodes> {nodes}</Nodes> \n <Properties> {properties}</Properties> \n <PotentialTriples> {potential_triples}</PotentialTriples></Ontology> \n"
    system_prompt += "You will solve the task by two steps: Step1(delimited by '<Step1>' and '</Step1>') Identify the appropriate semantic type and property for each column in the table. Step2(delimited by '<Step2>' and '</Step2>') Generate a semantic graph for the table. The solution are delimited by '<Solution>' and '</Solution>'.\n\
The examples of tasks are delimited by '<Examples>' and '</Examples>'. \n"
    examples = "<Examples>"
    for example_data in train_data:
        examples += f"<Table> {example_data['table']} </Table>\n"
        examples += "<Solution>\n"
        examples += f"<Step1> {example_data['semantic_graph']['SetSemanticType']} </Step1>\n"
        examples += f"<Step2> {example_data['semantic_graph']['SetInternalLink']} </Step2>\n"
        examples += "</Solution>"
    examples += "</Examples>"
    system_prompt += examples
# 2. Extract the column names and their corresponding row values from the lists of dictionaries. \n\
# 4. Rethink the internal links and the list of triples of semantic types and properties, change the unlogical part of internal links and list of triples of semantic types and properties. \n\
    chain_prompt_1 = "Step1 SetSemanticType: \n\
The system will first identify the appropriate semantic type and property for each column in the table. \n\
Solution Steps: \n\
1. Begin the response with 'Let's think step by step'. \n\
2. Following the reasoning steps, search the potential triples whose subject is start with  in the ontology to find the appropriate semantic types for each column in the table by reasoning the column name and the row values. All columns must be mapped to the Ontology. \
2. Following with reasoning steps, identify the appropriate semantic types from nodes in the ontology for each column in the table by reasoning the column name and the row values. All columns must be mapped to the Ontology. \
ONLY if all of the row values in the column are <Empty>, the system will take the column name into consideration to find the appropriate semantic types from nodes in the ontology. \n\
3. Find the appropriate properties from properties in the ontology to link the columns and the semantic types by reasoning the column names, the row values and the semantic types. \n\
4. The system will output the semantic types and properties in the format of a list of triples, where each triple has the form (subject, predicate, object). The subject is the semantic type, the predicate is the property, and the object is the column in the table.\
Please output the list of triples using <Step1> </Step1> tags in the format of in the Step1 of examples. For example: <Step1> [(semantic_type, property, column), (semantic_type, property, column), ...] </Step1>. "
    table  =f"<Table> {test_data} </Table>"
    chain_prompt_1 = chain_prompt_1 + table
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": chain_prompt_1}
    ]
    response = model.chat.completions.create(
    model="deepseek-chat",
    # model="gpt-3.5-turbo",
    messages=messages,
        stream=False
    )
    response_content = response.choices[0].message.content
    messages.append({"role": "assistant", "content": response_content})
    set_semantic_type = response_content.split("<Step1>")[1].split("</Step1>")[0]
    print(response_content)
#3. Review internal links and the list of triples of semantic types and properties, find the same semantic type which link to different internal nodes or the same internal node linked to different internal nodes. add a number to these semantic types and internal nodes to distinguish the different the nodes in the graph with the same name. \n\
    chain_prompt_2 = " Let us do Step2 given the table and Step1. \n\
Step2 Problem Statement: \n\
Given the list of triples of semantic types of the table (delimited by '<Step1>' and '</Step1>') and the original table, generate a semantic graph for the table. The semantic graph is a graph with nodes and edges. \
The nodes are the semantic types or internal nodes in the ontology, and the edges are the properties that link between the nodes. Hint: The graph is a tree structure.\n\
Solution: \n\
1. Begin the response with 'Let's think step by step'. \n\
2. Find the main root of the graph.  \n\
2. Following with reasoning steps, establish internal links between the semantic types based on the properties defined in the ontology and the nodes in the ontology. \n\
3. Review the internal links and the list of triples of semantic types and properties to find the semantic types which is isolated from the graph. Go back to Step1 to find the appropriate semantic types for these isolated semantic types to ensure all semantic types are connected in the graph and after that update the step1 solution. Hint: The graph is a tree structure. \n\
4. Output the final internal links using <Step2> </Step2> tag in the format of in the Step2 of examples. For example: <Step2> [(internal_node, property, internal_node), (internal_node, property, semantic_type), (internal_node, property, internal_node), ...] </Step2>.\n"
    chain_prompt_2 = chain_prompt_2 + f"<Step1> {set_semantic_type} </Step1>\n" + table
    messages2 = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": chain_prompt_2}
    ]
    response = model.chat.completions.create(
        model="deepseek-chat",
        messages=messages2,
        stream=False,
    )
    response_content = response.choices[0].message.content
    messages2.append({"role": "assistant", "content": response_content})
    return response_content, messages2

In [ ]:
from openai import OpenAI

model = OpenAI(api_key="")
res, mesg = generate_prompt_chain(train_data, test_data[0]['table'], model, ontology, True)
print(res)

In [ ]:
####Draft below###

In [6]:
for i, data in enumerate(test_data[:1]):
    message.append({"role": "user", "content": f"Table: {data['table']}. Please provide a step-by-step breakdown you do and detailed information about it."})   
    # prompt += f". The semantic graph of the table in the strict format of given examples above:"

In [7]:
text_file = open("Output.txt", "w")

text_file.write(str(message))

text_file.close()

In [ ]:
from openai import OpenAI

model = OpenAI(api_key="", base_url="https://api.deepseek.com")

response = model.chat.completions.create(
    model="deepseek-chat",
    messages=message,
    stream=False,
)
response_content = response.choices[0].message.content
text_file = open("response.txt", "w")

text_file.write(response_content)

text_file.close()
print(f"Messages Round 1: {response.choices[0].message.content}")


In [ ]:
message.append({"role": "user", "content": "I don't want to seperate the column into different SemanticType and do not ignore the columns whose value  <Empty>."})
response = model.chat.completions.create(
    model="deepseek-chat",
    messages=message
)

message.append(response.choices[0].message)
print(f"Messages Round 2: {response.choices[0].message.content}")

In [ ]:
message.append(response.choices[0].message)
message.append({"role": "user", "content": "Generate the semantic graph."})
response = model.chat.completions.create(
    model="deepseek-chat",
    messages=message
)
message.append(response.choices[0].message)
print(f"Messages Round 2: {response.choices[0].message.content}")

In [3]:
import json

def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def mean_reciprocal_rank(predicted, gold):
    ranks = []
    for item in gold:
        if item in predicted:
            rank = predicted.index(item) + 1
            ranks.append(1 / rank)
    return sum(ranks) / len(gold) if ranks else 0


    from sklearn.metrics import precision_score, recall_score

def calculate_precision_recall(gold_semantics, predicted_semantics):
    # Convert lists to sets for comparison
    gold_set = set(tuple(item) for item in gold_semantics)
    predicted_set = set(tuple(item) for item in predicted_semantics)

    # Calculate true positives, false positives, and false negatives
    true_positives = len(gold_set.intersection(predicted_set))
    false_positives = len(predicted_set - gold_set)
    false_negatives = len(gold_set - predicted_set)
    print(f"True Positives: {true_positives}")
    print(f"Intersection: {gold_set.intersection(predicted_set)}")

    # Calculate precision and recall
    precision = true_positives / (len(predicted_set)  - 1) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / len(gold_set) if (true_positives + false_negatives) > 0 else 0
    return precision, recall




def find_differences(list1, list2):
    set1 = set(tuple(item) for item in list1)
    set2 = set(tuple(item) for item in list2)
    differences = set1.symmetric_difference(set2)
    return [list(item) for item in differences]

# Load the JSON files
gold_semantics= load_json('Data/models-triples/s05-met-model.json')['semantic_triples']
predicted_semantics = load_json('s03_predict.json')['SetSemanticType']  # Replace with the actual path to the other file

precision, recall = calculate_precision_recall(gold_semantics, predicted_semantics)
print(f"Precision: {precision}")
print(f"Recall: {recall}")

# # Calculate MRR
# mrr = mean_reciprocal_rank(predicted_semantics, gold_semantics)
# print(f"Mean Reciprocal Rank (MRR): {mrr}")

# Find differences
differences = find_differences(predicted_semantics, gold_semantics)
print("Differences between the triples:")
for diff in differences:
    print(diff)

True Positives: 17
Intersection: {('crm:E52_Time-Span1', 'rdfs:label', 'Date'), ('crm:E33_Linguistic_Object4', 'rdf:value', 'Description'), ('crm:E82_Actor_Appellation1', 'rdf:value', 'Who'), ('crm:E33_Linguistic_Object2', 'rdf:value', 'Dimensions'), ('crm:E42_Identifier1', 'rdf:value', 'id'), ('crm:E52_Time-Span1', 'crm:P82a_begin_of_the_begin', 'date_earliest'), ('crm:E17_Type_Assignment1', 'karma:classLink', 'type_assignment_uri'), ('crm:E42_Identifier2', 'rdf:value', 'Accession Number'), ('crm:E38_Image1', 'karma:classLink', 'image'), ('crm:E12_Production1', 'karma:classLink', 'production_uri'), ('crm:E53_Place1', 'rdfs:label', 'Geography'), ('crm:E33_Linguistic_Object3', 'rdf:value', 'Credit Line'), ('crm:E33_Linguistic_Object5', 'rdf:value', 'Provenance'), ('crm:E22_Man-Made_Object1', 'karma:classLink', 'object_uri'), ('crm:E33_Linguistic_Object1', 'rdf:value', 'Medium'), ('crm:E52_Time-Span1', 'crm:P82b_end_of_the_end', 'date_latest'), ('crm:E39_Actor1', 'karma:classLink', 'arti

In [42]:
predicted_semantics.sort(key=lambda x: x[2])
gold_semantics.sort(key=lambda x: x[2]  )
predicted_semantics


[['crm:E22_Man-Made_Object1',
  'crm:P108i_was_produced_by',
  'crm:E12_Production1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P41i_was_classified_by',
  'crm:E17_Type_Assignment1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P67i_is_referred_to_by',
  'crm:E33_Linguistic_Object3'],
 ['crm:E22_Man-Made_Object1',
  'crm:P67i_is_referred_to_by',
  'crm:E33_Linguistic_Object4'],
 ['crm:E22_Man-Made_Object1',
  'crm:P67i_is_referred_to_by',
  'crm:E33_Linguistic_Object5'],
 ['crm:E22_Man-Made_Object1', 'crm:P102_has_title', 'crm:E35_Title1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P138i_has_representation',
  'crm:E38_Image1'],
 ['crm:E12_Production1', 'crm:P14_carried_out_by', 'crm:E39_Actor1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P1_is_identified_by',
  'crm:E42_Identifier1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P1_is_identified_by',
  'crm:E42_Identifier2'],
 ['crm:E12_Production1', 'crm:P4_has_time-span', 'crm:E52_Time-Span1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P53_has_former_or_current_loca

In [37]:
gold_semantics

[['crm:E22_Man-Made_Object1',
  'crm:P108i_was_produced_by',
  'crm:E12_Production1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P41i_was_classified_by',
  'crm:E17_Type_Assignment1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P67i_is_referred_to_by',
  'crm:E33_Linguistic_Object1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P67i_is_referred_to_by',
  'crm:E33_Linguistic_Object2'],
 ['crm:E22_Man-Made_Object1',
  'crm:P67i_is_referred_to_by',
  'crm:E33_Linguistic_Object3'],
 ['crm:E22_Man-Made_Object1',
  'crm:P67i_is_referred_to_by',
  'crm:E33_Linguistic_Object4'],
 ['crm:E22_Man-Made_Object1',
  'crm:P67i_is_referred_to_by',
  'crm:E33_Linguistic_Object5'],
 ['crm:E22_Man-Made_Object1',
  'crm:P138i_has_representation',
  'crm:E38_Image1'],
 ['crm:E12_Production1', 'crm:P14_carried_out_by', 'crm:E39_Actor1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P1_is_identified_by',
  'crm:E42_Identifier1'],
 ['crm:E22_Man-Made_Object1',
  'crm:P1_is_identified_by',
  'crm:E42_Identifier2'],
 ['crm:E12_Productio

In [ ]:
    {
        "id":10000831,
        "image":"http:\/\/images.metmuseum.org\/CRDImages\/ad\/web-large\/DP104414.jpg",
        "Date":"1700-1710",
        "date_latest":"",
        "date_earliest":"",
        "Geography":"Mid-Atlantic, New York City, New York, United States",
        "Medium":"Silver",
        "Dimensions":"\n",
        "Classification":"Silver",
        "Credit Line":"Samuel D. Lee Fund, 1938",
        "Accession Number":"38.63",
        "object_uri":"http:\/\/www.metmuseum.org\/object\/38.63",
        "type_assignment_uri":"http:\/\/www.metmuseum.org\/object\/38.63\/work_type",
        "production_uri":"http:\/\/www.metmuseum.org\/object\/38.63\/production",
        "Who":"Cornelius Kierstede",
        "artist_uri":"http:\/\/www.metmuseum.org\/person\/cornelius-kierstede",
        "What":[
            "Metal",
            "Silver",
            "Bowls"
        ],
        "Description":"Derived from Dutch, Scandinavian, and English sources, brandywine bowls were used on festive occasions such as the kindermaal, when women gathered to welcome a newborn child. Following Dutch custom, a bowl would be filled with raisins and brandy and passed from guest to guest. Here, boldly chased tulips create a sumptuous effect. The bowl belonged to Theunis Jacobsen Quick, a wealthy baker, and his wife, Vroutje 4516 Janse Haring.",
        "Provenance":"\n"
    },